# Machine Learning Programming Assignment 1:
## KNN algorithm implement from scratch with 10-fold cross validation.
## Datasets used: 1.) Iris dataset (implemented to campare with reference)
##                            2.) Car Evaluation Dataset
##                            3.) Breast Cancer Dataset
##                            2.) Hayes-Roth Dataset

In [39]:
#libraries used for our assignments are: 
from random import seed
from random import randrange
from csv import reader
from math import sqrt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix,classification_report

#### Across the program, we will be using camel Notations for Functions/ Methods & Variables/Paraments

#### We read the CSV/Data files in 2 format:
##### 1.) By using the below function, named load_csv. The results are stored in a list
##### 2.) By reading the CSV using pandas, which gives us the result in a dataframe. We will then convert into a list

In [40]:
#Method to load the file and return the dataset available in the file ----------> First Visisted
def loadFile(filename):
    dataset = list()
    with open(filename, 'r') as file:
        data = reader(file)
        for row in data:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [41]:
# Let us check whether the function works: 
seed(1)
filename = 'iris.csv'
dataset = loadFile(filename)
dataset

[['5.1', '3.5', '1.4', '0.2', 'Iris-setosa'],
 ['4.9', '3.0', '1.4', '0.2', 'Iris-setosa'],
 ['4.7', '3.2', '1.3', '0.2', 'Iris-setosa'],
 ['4.6', '3.1', '1.5', '0.2', 'Iris-setosa'],
 ['5.0', '3.6', '1.4', '0.2', 'Iris-setosa'],
 ['5.4', '3.9', '1.7', '0.4', 'Iris-setosa'],
 ['4.6', '3.4', '1.4', '0.3', 'Iris-setosa'],
 ['5.0', '3.4', '1.5', '0.2', 'Iris-setosa'],
 ['4.4', '2.9', '1.4', '0.2', 'Iris-setosa'],
 ['4.9', '3.1', '1.5', '0.1', 'Iris-setosa'],
 ['5.4', '3.7', '1.5', '0.2', 'Iris-setosa'],
 ['4.8', '3.4', '1.6', '0.2', 'Iris-setosa'],
 ['4.8', '3.0', '1.4', '0.1', 'Iris-setosa'],
 ['4.3', '3.0', '1.1', '0.1', 'Iris-setosa'],
 ['5.8', '4.0', '1.2', '0.2', 'Iris-setosa'],
 ['5.7', '4.4', '1.5', '0.4', 'Iris-setosa'],
 ['5.4', '3.9', '1.3', '0.4', 'Iris-setosa'],
 ['5.1', '3.5', '1.4', '0.3', 'Iris-setosa'],
 ['5.7', '3.8', '1.7', '0.3', 'Iris-setosa'],
 ['5.1', '3.8', '1.5', '0.3', 'Iris-setosa'],
 ['5.4', '3.4', '1.7', '0.2', 'Iris-setosa'],
 ['5.1', '3.7', '1.5', '0.4', 'Iri

#### We will be using the same methods for all 4 datasets, apart from the following:
##### 1.) stringToFloat -----> Used on the dataset columns to change the datatype from string to float inorder to fit in the model.
##### 2.) stringToInt --------> Used on the dataset columns to change the datatype from string to int inorder to fit in the model.  (mostly the last column i.e. the "to be predicted column")

In [42]:
# Method to the dataset type  from string to float ------> 2nd visited
def stringToFloat(dataset, column):
    try:
        for row in dataset:
            row[column] = float(row[column].strip())
    except:
        pass

In [43]:
# Method to the dataset type  from string to float ------> 3rd visited
def stringToInt(dataset, column):
    predictColumn = [row[column] for row in dataset]
    uniqueColumn = set(predictColumn) # set converts to unique values
    result = dict()
    for i, value in enumerate(uniqueColumn): # We use enumerate to get the value from the dataset and create an interget value accordingly
        result[value] = i
    for row in dataset:
        row[column] = result[row[column]]
    return result

#### Let us now create a method which help us in the process the k-fold cross validation on the dataset

In [44]:
# 5th visited
def kFoldCrossValidation(dataset, n):
    datasetSplit = list()
    tempDataset = list(dataset)
    segmentationSize = int(len(dataset) / n)
    for _ in range(n):
        fold = list()
        while len(fold) < segmentationSize:
            index = randrange(len(tempDataset)) # randomly taking a index and appending into a fold list
            fold.append(tempDataset.pop(index)) 
        datasetSplit.append(fold) #appending the fold to dataset and iterating for other folds
    return datasetSplit

####  Method for finding calculating accuracy metrix value:

In [45]:
# 10th visited
def accuracyMetric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

#### The below method does most of the work:
#### Firstly, calls the cross validation methods and gets the result
#### We fetch and convert the training/ testing test to fit in out model appropriately
#### We call the accuracyMetric function and get the final score of the algorithm

In [46]:
# Evaluate an algorithm using a cross validation split ----> 4th visited
def evaluationMethod(dataset, algorithm, n_folds, num_neighbors,d):
    folds = kFoldCrossValidation(dataset, n_folds) #gets the dataset with folds
    scores = list()
    for fold in folds:
        trainSet = list(folds)
        trainSet.remove(fold)
        trainSet = sum(trainSet, []) #removing a [bracket] i.e. making a 3D list -> 2D list
        testSet = list()
        for row in fold:
            rowCopy = list(row)
            testSet.append(rowCopy)
            rowCopy[-1] = None
        predicted = algorithm(trainSet, testSet, num_neighbors,d)
        actual = [row[-1] for row in fold]
        accuracy = accuracyMetric(actual, predicted)
        #confusionMatrix(actual, predicted)
        scores.append(accuracy)
    return scores

#### If the given value for D equals to 1 --> then we find the distance using Euclidean Distance

In [47]:
# Calculate the Euclidean distance between two vectors --------> 9th visted
def euclideanDistance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

#### If the given value for D equals to 2 --> then we find the distance using Hamming Distance

In [48]:
def hammingDistance(row1,row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance+= not(row1[i]==row2[i])
    return distance

#### If the given value for D equals to 3--> then we find the distance using Manhattan Distance

In [49]:
def manhattanDistance(row1,row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += abs(row1[i] - row2[i]) 
    return distance

In [50]:
# Locate the most similar neighbors ---------------------> 8th visited
def getNeighbors(train, test_row, num_neighbors,d):
    distances = list()
    for train_row in train:
        if(d==1): # value of d is given while calling evaluationMethod
            dist = euclideanDistance(test_row, train_row)
        if(d==2):
            dist = hammingDistance(test_row, train_row)
        if(d==3):
            dist = manhattanDistance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

In [51]:
# Make a prediction with neighbors -------------------------> 7th visited
def predictClassification(train, test_row, num_neighbors,d):
    neighbors = getNeighbors(train, test_row, num_neighbors,d)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction

In [52]:
# KNN Algorithm -------------------------------------------> 6th visited
def kNearestNeighbors(train, test, num_neighbors,d):
    predictions = list()
    for row in test:
        output = predictClassification(train, row, num_neighbors,d)
        predictions.append(output)
    return(predictions)

In [53]:
def confusionMatrix(Y_test,predictions):
    print("\nConfusion Matrix:\n\n",confusion_matrix(Y_test,predictions))

## Letz do KNN for all the 4 dataset using Euclidean Distance

In [54]:
# Test the kNN on the Iris Flowers dataset
seed(1)
filename = 'iris.csv'
dataset = loadFile(filename)
for i in range(len(dataset[0])-1):
    stringToFloat(dataset, i)
# convert class column to integers
stringToInt(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 10
d = 1
print("For tuning the KNN value with different neighbour value:")
print("")

print("\033[1mIris Flower dataset for 10-fold cross validation with neighbours as 3:\033[0m")
scores = evaluationMethod(dataset, kNearestNeighbors, n_folds, 3,d)
print('\033[1mScores:\033[0m %s' % scores)
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("\033[1mIris Flower dataset for 10-fold cross validation with neighbours as 9:\033[0m")
scores = evaluationMethod(dataset, kNearestNeighbors, n_folds, 9,d)
print('\033[1mScores: \033[0m \033 %s' % scores)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mIris Flower dataset for 10-fold cross validation with neighbours as 7:")
scores = evaluationMethod(dataset, kNearestNeighbors, n_folds, 7,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("")
print("Therefore, the prediction accuracy is the best for neighbour value =\033[1m 7 \033[0m")

For tuning the KNN value with different neighbour value:

Iris Flower dataset for 10-fold cross validation with neighbours as 3:
Scores: [93.33333333333333, 93.33333333333333, 93.33333333333333, 100.0, 100.0, 100.0, 93.33333333333333, 93.33333333333333, 93.33333333333333, 100.0]
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
Mean Accuracy: 96.000%
Iris Flower dataset for 10-fold cross validation with neighbours as 9:
Scores:   [93.33333333333333, 86.66666666666667, 100.0, 93.33333333333333, 93.33333333333333, 100.0, 93.33333333333333, 100.0, 93.33333333333333, 100.0]
Mean Accuracy: 95.333%
-------------------------------------------------------------------------------------------------------------------------------
-----------------------------------------------------------------

In [55]:
#Test the KNN on Breast Cancer Dataset
filenameCancer = "breast-cancer.data"
datasetCancer = loadFile(filenameCancer)
for i in range(len(datasetCancer[0])-1):
    stringToFloat(datasetCancer, i)
for index in range(len(datasetCancer[0])):
    if(type(datasetCancer[0][index])==str):
        stringToInt(datasetCancer,index)
# We deleted 2nd and 3rd column for better prediction

d = 1
n_folds = 10
print("For tuning the KNN value with different neighbour value:")
print("")
print("\033[1mBreat Cancer Dataset for 10-fold cross validation with neighbours as 3:\033[0m")
scores = evaluationMethod(datasetCancer, kNearestNeighbors, n_folds, 3,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy\033[0m: %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mBreat Cancer Dataset for 10-fold cross validation with neighbours as 7:\033[0m")
scores = evaluationMethod(datasetCancer, kNearestNeighbors, n_folds, 7,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mBreat Cancer Dataset for 10-fold cross validation with neighbours as 9:\033[0m")
scores = evaluationMethod(datasetCancer, kNearestNeighbors, n_folds, 9,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))

print("")
print("Therefore, the prediction accuracy is the best for neighbour value =\033[1m 9 \033[0m")

For tuning the KNN value with different neighbour value:

Breat Cancer Dataset for 10-fold cross validation with neighbours as 3:
Scores: [71.42857142857143, 82.14285714285714, 75.0, 71.42857142857143, 82.14285714285714, 75.0, 82.14285714285714, 78.57142857142857, 67.85714285714286, 75.0]
ean Accuracy: 76.071%
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
Breat Cancer Dataset for 10-fold cross validation with neighbours as 7:
Scores: [64.28571428571429, 78.57142857142857, 75.0, 85.71428571428571, 71.42857142857143, 78.57142857142857, 78.57142857142857, 71.42857142857143, 78.57142857142857, 64.28571428571429]
ean Accuracy: 74.643%
-------------------------------------------------------------------------------------------------------------------------------
------------------------

In [56]:
#Test the KNN on Car Evaluation Dataset
filenameCar = "car.data"
datasetCar = loadFile(filenameCar)
# We dont need to convert dataset into Float for this dataset
# for i in range(len(datasetCar[0])-1):
#     stringToFloat(datasetCar, i)
for index in range(len(datasetCar[0])):
    if(type(datasetCar[0][index])==str):
        stringToInt(datasetCar,index)
        
for j in datasetCar:
    del j[2]
    del j[3]
    
n_folds = 10
d = 1
print("For tuning the KNN value with different neighbour value:")
print("")
print("\033[1mCar Evaluation Dataset for 10-fold cross validation with neighbours as 3:\033[0m")
scores = evaluationMethod(datasetCar, kNearestNeighbors, n_folds, 3,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mCar Evaluation Dataset for 10-fold cross validation with neighbours as 7:\033[0m")
scores = evaluationMethod(datasetCar, kNearestNeighbors, n_folds, 7,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mCar Evaluation Dataset for 10-fold cross validation with neighbours as 9:\033[0m")
scores = evaluationMethod(datasetCar, kNearestNeighbors, n_folds, 9,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))

print("")
print("Therefore, the prediction accuracy is the best for neighbour value =\033[1m 7 \033[0m")

For tuning the KNN value with different neighbour value:

Car Evaluation Dataset for 10-fold cross validation with neighbours as 3:
Scores: [91.27906976744185, 84.30232558139535, 87.79069767441861, 86.62790697674419, 88.37209302325581, 85.46511627906976, 83.72093023255815, 86.04651162790698, 89.53488372093024, 89.53488372093024]
ean Accuracy: 87.267%
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
Car Evaluation Dataset for 10-fold cross validation with neighbours as 7:
Scores: [90.69767441860465, 83.72093023255815, 84.88372093023256, 81.97674418604652, 82.55813953488372, 86.04651162790698, 84.88372093023256, 88.37209302325581, 87.79069767441861, 86.04651162790698]
ean Accuracy: 85.698%
------------------------------------------------------------------------------------------------

In [57]:
# Test the KNN on for hayes-roth Dataset
filenameHayes = 'hayes-roth.data'
datasetHayes = loadFile(filenameHayes)
n_folds = 10
num_neighbors = 9
for i in range(len(datasetHayes[0])-1):
    stringToFloat(datasetHayes, i)
for index in range(len(datasetHayes[0])):
    if(type(datasetHayes[0][index])==str):
        stringToInt(datasetHayes,index)
for j in datasetHayes:
    del j[0]
    
print("For tuning the KNN value with different neighbour value:")
print("")
print("\033[1mHayes-roth Dataset for 10-fold cross validation with neighbours as 3:\033[0m")
scores4 = evaluationMethod(datasetHayes, kNearestNeighbors, n_folds, 3,d)
print('\033[1mScores:\033[0m %s' % scores4)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores4)/float(len(scores4))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mHayes-roth Dataset for 10-fold cross validation with neighbours as 5:\033[0m")
scores4 = evaluationMethod(datasetHayes, kNearestNeighbors, n_folds, 5,d)
print('\033[1mScores:\033[0m %s' % scores4)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores4)/float(len(scores4))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mHayes-roth Dataset for 10-fold cross validation with neighbours as 9:\033[0m")
scores4 = evaluationMethod(datasetHayes, kNearestNeighbors, n_folds, 9,d)
print('\033[1mScores:\033[0m %s' % scores4)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores4)/float(len(scores4))))


print("")
print("Therefore, the prediction accuracy is the best for neighbour value =\033[1m 3 \033[0m")

For tuning the KNN value with different neighbour value:

Hayes-roth Dataset for 10-fold cross validation with neighbours as 3:
Scores: [69.23076923076923, 69.23076923076923, 61.53846153846154, 76.92307692307693, 76.92307692307693, 69.23076923076923, 46.15384615384615, 46.15384615384615, 46.15384615384615, 84.61538461538461]
Mean Accuracy: 64.615%
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
Hayes-roth Dataset for 10-fold cross validation with neighbours as 5:
Scores: [61.53846153846154, 76.92307692307693, 53.84615384615385, 84.61538461538461, 69.23076923076923, 84.61538461538461, 30.76923076923077, 61.53846153846154, 76.92307692307693, 61.53846153846154]
Mean Accuracy: 66.154%
------------------------------------------------------------------------------------------------------

## Letz do KNN for all the 4 dataset using Hamming Distance

In [58]:
# Test the kNN on the Iris Flowers dataset
seed(1)
filename = 'iris.csv'
dataset = loadFile(filename)
for i in range(len(dataset[0])-1):
    stringToFloat(dataset, i)
# convert class column to integers
stringToInt(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 10
d = 2
print("For tuning the KNN value with different neighbour value:")
print("")

print("\033[1mIris Flower dataset for 10-fold cross validation with neighbours as 3:\033[0m")
scores = evaluationMethod(dataset, kNearestNeighbors, n_folds, 3,d)
print('\033[1mScores:\033[0m %s' % scores)
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("\033[1mIris Flower dataset for 10-fold cross validation with neighbours as 9:\033[0m")
scores = evaluationMethod(dataset, kNearestNeighbors, n_folds, 9,d)
print('\033[1mScores: \033[0m \033 %s' % scores)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mIris Flower dataset for 10-fold cross validation with neighbours as 7:")
scores = evaluationMethod(dataset, kNearestNeighbors, n_folds, 7,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("")
print("Therefore, the prediction accuracy is the best for neighbour value =\033[1m 7 \033[0m")

For tuning the KNN value with different neighbour value:

Iris Flower dataset for 10-fold cross validation with neighbours as 3:
Scores: [73.33333333333333, 80.0, 86.66666666666667, 80.0, 86.66666666666667, 93.33333333333333, 93.33333333333333, 73.33333333333333, 80.0, 100.0]
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
Mean Accuracy: 84.667%
Iris Flower dataset for 10-fold cross validation with neighbours as 9:
Scores:   [73.33333333333333, 86.66666666666667, 86.66666666666667, 86.66666666666667, 86.66666666666667, 73.33333333333333, 66.66666666666666, 100.0, 100.0, 86.66666666666667]
Mean Accuracy: 84.667%
-------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------

In [59]:
#Test the KNN on Breast Cancer Dataset
filenameCancer = "breast-cancer.data"
datasetCancer = loadFile(filenameCancer)
for i in range(len(datasetCancer[0])-1):
    stringToFloat(datasetCancer, i)
for index in range(len(datasetCancer[0])):
    if(type(datasetCancer[0][index])==str):
        stringToInt(datasetCancer,index)
d = 2
n_folds = 10
print("For tuning the KNN value with different neighbour value:")
print("")
print("\033[1mBreat Cancer Dataset for 10-fold cross validation with neighbours as 3:\033[0m")
scores = evaluationMethod(datasetCancer, kNearestNeighbors, n_folds, 3,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy\033[0m: %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mBreat Cancer Dataset for 10-fold cross validation with neighbours as 7:\033[0m")
scores = evaluationMethod(datasetCancer, kNearestNeighbors, n_folds, 7,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mBreat Cancer Dataset for 10-fold cross validation with neighbours as 9:\033[0m")
scores = evaluationMethod(datasetCancer, kNearestNeighbors, n_folds, 9,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))

print("")
print("Therefore, the prediction accuracy is the best for neighbour value =\033[1m 9 \033[0m")

For tuning the KNN value with different neighbour value:

Breat Cancer Dataset for 10-fold cross validation with neighbours as 3:
Scores: [71.42857142857143, 82.14285714285714, 75.0, 78.57142857142857, 71.42857142857143, 75.0, 82.14285714285714, 67.85714285714286, 71.42857142857143, 75.0]
ean Accuracy: 75.000%
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
Breat Cancer Dataset for 10-fold cross validation with neighbours as 7:
Scores: [60.71428571428571, 85.71428571428571, 78.57142857142857, 78.57142857142857, 78.57142857142857, 78.57142857142857, 82.14285714285714, 71.42857142857143, 71.42857142857143, 71.42857142857143]
ean Accuracy: 75.714%
-------------------------------------------------------------------------------------------------------------------------------
-----------

In [60]:
#Test the KNN on Car Evaluation Dataset
filenameCar = "car.data"
datasetCar = loadFile(filenameCar)
# We dont need to convert dataset into Float for this dataset
# for i in range(len(datasetCar[0])-1):
#     stringToFloat(datasetCar, i)
for index in range(len(datasetCar[0])):
    if(type(datasetCar[0][index])==str):
        stringToInt(datasetCar,index)

for j in datasetCar:
    del j[2]
    del j[3]
    
n_folds = 10
d = 2
print("For tuning the KNN value with different neighbour value:")
print("")
print("\033[1mCar Evaluation Dataset for 10-fold cross validation with neighbours as 3:\033[0m")
scores = evaluationMethod(datasetCar, kNearestNeighbors, n_folds, 3,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mCar Evaluation Dataset for 10-fold cross validation with neighbours as 7:\033[0m")
scores = evaluationMethod(datasetCar, kNearestNeighbors, n_folds, 7,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mCar Evaluation Dataset for 10-fold cross validation with neighbours as 9:\033[0m")
scores = evaluationMethod(datasetCar, kNearestNeighbors, n_folds, 9,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))

print("")
print("Therefore, the prediction accuracy is the best for neighbour value =\033[1m 7 \033[0m")

For tuning the KNN value with different neighbour value:

Car Evaluation Dataset for 10-fold cross validation with neighbours as 3:
Scores: [91.27906976744185, 84.30232558139535, 87.79069767441861, 86.62790697674419, 88.37209302325581, 85.46511627906976, 83.72093023255815, 86.04651162790698, 89.53488372093024, 89.53488372093024]
ean Accuracy: 87.267%
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
Car Evaluation Dataset for 10-fold cross validation with neighbours as 7:
Scores: [90.69767441860465, 83.72093023255815, 84.88372093023256, 81.97674418604652, 82.55813953488372, 86.04651162790698, 84.88372093023256, 88.37209302325581, 87.79069767441861, 86.04651162790698]
ean Accuracy: 85.698%
------------------------------------------------------------------------------------------------

In [61]:
# Test the KNN on for hayes-roth Dataset
filenameHayes = 'hayes-roth.data'
datasetHayes = loadFile(filenameHayes)
n_folds = 10
num_neighbors = 9
for i in range(len(datasetHayes[0])-1):
    stringToFloat(datasetHayes, i)
for index in range(len(datasetHayes[0])):
    if(type(datasetHayes[0][index])==str):
        stringToInt(datasetHayes,index)
for j in datasetHayes:
    del j[0]
d = 2
print("For tuning the KNN value with different neighbour value:")
print("")
print("\033[1mHayes-roth Dataset for 10-fold cross validation with neighbours as 3:\033[0m")
scores4 = evaluationMethod(datasetHayes, kNearestNeighbors, n_folds, 3,d)
print('\033[1mScores:\033[0m %s' % scores4)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores4)/float(len(scores4))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mHayes-roth Dataset for 10-fold cross validation with neighbours as 5:\033[0m")
scores4 = evaluationMethod(datasetHayes, kNearestNeighbors, n_folds, 5,d)
print('\033[1mScores:\033[0m %s' % scores4)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores4)/float(len(scores4))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mHayes-roth Dataset for 10-fold cross validation with neighbours as 9:\033[0m")
scores4 = evaluationMethod(datasetHayes, kNearestNeighbors, n_folds, 9,d)
print('\033[1mScores:\033[0m %s' % scores4)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores4)/float(len(scores4))))


print("")
print("Therefore, the prediction accuracy is the best for neighbour value =\033[1m 3 \033[0m")

For tuning the KNN value with different neighbour value:

Hayes-roth Dataset for 10-fold cross validation with neighbours as 3:
Scores: [61.53846153846154, 46.15384615384615, 38.46153846153847, 76.92307692307693, 69.23076923076923, 76.92307692307693, 38.46153846153847, 53.84615384615385, 38.46153846153847, 76.92307692307693]
Mean Accuracy: 57.692%
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
Hayes-roth Dataset for 10-fold cross validation with neighbours as 5:
Scores: [53.84615384615385, 76.92307692307693, 84.61538461538461, 53.84615384615385, 69.23076923076923, 53.84615384615385, 38.46153846153847, 69.23076923076923, 69.23076923076923, 61.53846153846154]
Mean Accuracy: 63.077%
------------------------------------------------------------------------------------------------------

## Letz do KNN for all the 4 dataset using Manhattan Distance

In [62]:
# Test the kNN on the Iris Flowers dataset
seed(1)
filename = 'iris.csv'
dataset = loadFile(filename)
for i in range(len(dataset[0])-1):
    stringToFloat(dataset, i)
# convert class column to integers
stringToInt(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 10
d = 3
print("For tuning the KNN value with different neighbour value:")
print("")

print("\033[1mIris Flower dataset for 10-fold cross validation with neighbours as 3:\033[0m")
scores = evaluationMethod(dataset, kNearestNeighbors, n_folds, 3,d)
print('\033[1mScores:\033[0m %s' % scores)
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("\033[1mIris Flower dataset for 10-fold cross validation with neighbours as 9:\033[0m")
scores = evaluationMethod(dataset, kNearestNeighbors, n_folds, 9,d)
print('\033[1mScores: \033[0m \033 %s' % scores)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mIris Flower dataset for 10-fold cross validation with neighbours as 7:")
scores = evaluationMethod(dataset, kNearestNeighbors, n_folds, 7,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("")
print("Therefore, the prediction accuracy is the best for neighbour value =\033[1m 9 \033[0m")

For tuning the KNN value with different neighbour value:

Iris Flower dataset for 10-fold cross validation with neighbours as 3:
Scores: [93.33333333333333, 93.33333333333333, 93.33333333333333, 100.0, 100.0, 100.0, 93.33333333333333, 93.33333333333333, 93.33333333333333, 100.0]
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
Mean Accuracy: 96.000%
Iris Flower dataset for 10-fold cross validation with neighbours as 9:
Scores:   [93.33333333333333, 86.66666666666667, 100.0, 86.66666666666667, 93.33333333333333, 100.0, 93.33333333333333, 93.33333333333333, 93.33333333333333, 100.0]
Mean Accuracy: 94.000%
-------------------------------------------------------------------------------------------------------------------------------
-----------------------------------------------------

In [63]:
#Test the KNN on Breast Cancer Dataset
filenameCancer = "breast-cancer.data"
datasetCancer = loadFile(filenameCancer)
for i in range(len(datasetCancer[0])-1):
    stringToFloat(datasetCancer, i)
for index in range(len(datasetCancer[0])):
    if(type(datasetCancer[0][index])==str):
        stringToInt(datasetCancer,index)
d = 3
n_folds = 10
print("For tuning the KNN value with different neighbour value:")
print("")
print("\033[1mBreat Cancer Dataset for 10-fold cross validation with neighbours as 3:\033[0m")
scores = evaluationMethod(datasetCancer, kNearestNeighbors, n_folds, 3,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy\033[0m: %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mBreat Cancer Dataset for 10-fold cross validation with neighbours as 7:\033[0m")
scores = evaluationMethod(datasetCancer, kNearestNeighbors, n_folds, 7,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mBreat Cancer Dataset for 10-fold cross validation with neighbours as 9:\033[0m")
scores = evaluationMethod(datasetCancer, kNearestNeighbors, n_folds, 9,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))

print("")
print("Therefore, the prediction accuracy is the best for neighbour value =\033[1m 7 \033[0m")

For tuning the KNN value with different neighbour value:

Breat Cancer Dataset for 10-fold cross validation with neighbours as 3:
Scores: [71.42857142857143, 75.0, 71.42857142857143, 75.0, 78.57142857142857, 78.57142857142857, 82.14285714285714, 75.0, 67.85714285714286, 75.0]
ean Accuracy: 75.000%
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
Breat Cancer Dataset for 10-fold cross validation with neighbours as 7:
Scores: [64.28571428571429, 82.14285714285714, 75.0, 82.14285714285714, 75.0, 75.0, 78.57142857142857, 75.0, 75.0, 64.28571428571429]
ean Accuracy: 74.643%
-------------------------------------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------

In [64]:
#Test the KNN on Car Evaluation Dataset
filenameCar = "car.data"
datasetCar = loadFile(filenameCar)
# We dont need to convert dataset into Float for this dataset
# for i in range(len(datasetCar[0])-1):
#     stringToFloat(datasetCar, i)
for index in range(len(datasetCar[0])):
    if(type(datasetCar[0][index])==str):
        stringToInt(datasetCar,index)
for j in datasetCar:
    del j[2]
    del j[3]
n_folds = 10
d = 3
print("For tuning the KNN value with different neighbour value:")
print("")
print("\033[1mCar Evaluation Dataset for 10-fold cross validation with neighbours as 3:\033[0m")
scores = evaluationMethod(datasetCar, kNearestNeighbors, n_folds, 3,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mCar Evaluation Dataset for 10-fold cross validation with neighbours as 7:\033[0m")
scores = evaluationMethod(datasetCar, kNearestNeighbors, n_folds, 7,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mCar Evaluation Dataset for 10-fold cross validation with neighbours as 9:\033[0m")
scores = evaluationMethod(datasetCar, kNearestNeighbors, n_folds, 9,d)
print('\033[1mScores:\033[0m %s' % scores)
print('\033[Mean Accuracy:\033[0m %.3f%%' % (sum(scores)/float(len(scores))))

print("")
print("Therefore, the prediction accuracy is the best for neighbour value =\033[1m 7 \033[0m")

For tuning the KNN value with different neighbour value:

Car Evaluation Dataset for 10-fold cross validation with neighbours as 3:
Scores: [91.27906976744185, 84.30232558139535, 87.79069767441861, 86.62790697674419, 88.37209302325581, 85.46511627906976, 83.72093023255815, 86.04651162790698, 89.53488372093024, 89.53488372093024]
ean Accuracy: 87.267%
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
Car Evaluation Dataset for 10-fold cross validation with neighbours as 7:
Scores: [90.69767441860465, 83.72093023255815, 84.88372093023256, 81.97674418604652, 82.55813953488372, 86.04651162790698, 84.88372093023256, 88.37209302325581, 87.79069767441861, 86.04651162790698]
ean Accuracy: 85.698%
------------------------------------------------------------------------------------------------

In [65]:
# Test the KNN on for hayes-roth Dataset
filenameHayes = 'hayes-roth.data'
datasetHayes = loadFile(filenameHayes)
n_folds = 10
num_neighbors = 9
for i in range(len(datasetHayes[0])-1):
    stringToFloat(datasetHayes, i)
for index in range(len(datasetHayes[0])):
    if(type(datasetHayes[0][index])==str):
        stringToInt(datasetHayes,index)

for j in datasetHayes:
    del j[0]
d = 3
print("For tuning the KNN value with different neighbour value:")
print("")
print("\033[1mHayes-roth Dataset for 10-fold cross validation with neighbours as 3:\033[0m")
scores4 = evaluationMethod(datasetHayes, kNearestNeighbors, n_folds, 3,d)
print('\033[1mScores:\033[0m %s' % scores4)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores4)/float(len(scores4))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mHayes-roth Dataset for 10-fold cross validation with neighbours as 5:\033[0m")
scores4 = evaluationMethod(datasetHayes, kNearestNeighbors, n_folds, 5,d)
print('\033[1mScores:\033[0m %s' % scores4)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores4)/float(len(scores4))))
print("-------------------------------------------------------------------------------------------------------------------------------")
print("-------------------------------------------------------------------------------------------------------------------------------")
print("\033[1mHayes-roth Dataset for 10-fold cross validation with neighbours as 9:\033[0m")
scores4 = evaluationMethod(datasetHayes, kNearestNeighbors, n_folds, 9,d)
print('\033[1mScores:\033[0m %s' % scores4)
print('\033[1mMean Accuracy:\033[0m %.3f%%' % (sum(scores4)/float(len(scores4))))


print("")
print("Therefore, the prediction accuracy is the best for neighbour value =\033[1m 3 \033[0m")

For tuning the KNN value with different neighbour value:

Hayes-roth Dataset for 10-fold cross validation with neighbours as 3:
Scores: [69.23076923076923, 69.23076923076923, 61.53846153846154, 76.92307692307693, 76.92307692307693, 69.23076923076923, 38.46153846153847, 38.46153846153847, 46.15384615384615, 84.61538461538461]
Mean Accuracy: 63.077%
-------------------------------------------------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------------------------------------------------
Hayes-roth Dataset for 10-fold cross validation with neighbours as 5:
Scores: [76.92307692307693, 76.92307692307693, 61.53846153846154, 84.61538461538461, 69.23076923076923, 84.61538461538461, 38.46153846153847, 61.53846153846154, 76.92307692307693, 61.53846153846154]
Mean Accuracy: 69.231%
------------------------------------------------------------------------------------------------------